In [1]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

import sys
sys.path.append("../")
import os

import re
import datetime
import pandas as pd
import pandas_market_calendars as mcal
from polygon import RESTClient
#from tqdm.notebook import tqdm
import time
import pytz
eastern = 'US/Eastern'

In [6]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_split_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_splits(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting split data for "+ str(stock))
            return []

def get_dividend_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_dividends(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting dividend data for "+ str(stock))
            return []
        
def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified

In [31]:
get_folder_list()

['14-04-2021_t-16.14_pv_adv_4_2']

In [32]:
def get_folder_list():    
    path = '.'
    directory_contents = os.listdir(path)

    r = re.compile(".*pv_adv_[4]_[2]")
    folder_list = list(filter(r.match, directory_contents)) # Read Note
    return folder_list

def get_all_files_in_folder(folder):
    return os.listdir(folder)

def extract_date_from_filename(filename):
    split_a = filename.split('trades-')
    split_b = split_a[1].split('.')
    return split_b[0]

def update_all_files():
    a = get_folder_list()
    count = 0
    
    all_file_dfs = []
    
    for folder in tqdm(a, desc="folders"):
        all_files = get_all_files_in_folder(folder)
        count = count + 1
        
        for file in tqdm(all_files, desc="Folder Files"):
            file_date = extract_date_from_filename(file)
            file_df = pd.read_csv(folder+'/'+str(file))
            file_df['trade_date'] = file_date
            file_df['group'] = count
            file_df['random_id'] = file_df[['trade_date', 'symbol']].sum(axis=1).map(hash)            

            file_df.set_index("random_id", inplace=True)
            
            all_file_dfs.append(file_df)
            
    result_1 = pd.DataFrame().append(all_file_dfs)
    return result_1

def update_all_etf_files():

    all_file_dfs = []    
    for file in tqdm(all_etf_files, desc="etf Files"):
        file_date = extract_date_from_filename(file)
        file_df = pd.read_csv("pv_adv_major_etfs"+'/'+str(file))
        file_df['trade_date'] = file_date
        file_df['random_id'] = file_df[['trade_date', 'symbol']].sum(axis=1).map(hash)            
        file_df.set_index("random_id", inplace=True)

        all_file_dfs.append(file_df)
            
    result_1 = pd.DataFrame().append(all_file_dfs)
    return result_1

In [25]:
main_df = pd.read_csv('stocks-profile.csv')
q_sym_list = []
for i in range(5):
    q_sym = extract_qualified_symbols(main_df, i, 2)
    q_sym_list = q_sym_list + list(q_sym['symbol'])

In [26]:
splits_df = []
dividend_df = []

In [27]:
for i in tqdm(range(len(q_sym_list))):
    
    ticker = q_sym_list[i]
    
    sp = get_split_data_for_stock(ticker)
    div = get_dividend_data_for_stock(ticker)
    a = pd.DataFrame(sp)
    b = pd.DataFrame(div)
   
    splits_df.append(a)
    
    dividend_df.append(b)
    
    time.sleep(0.1)

In [28]:
dividend_df = pd.concat(dividend_df)
splits_df = pd.concat(splits_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [33]:
splits_df.head()

,declaredDate,exDate,forfactor,paymentDate,ratio,ticker,tofactor
0,2016-11-09,2016-11-09,1.0,2016-11-09,3.00,GNUS,3.0
0,NaN,2000-04-11,NaN,2000-04-11,0.25,NOK,NaN
1,NaN,1999-04-12,NaN,1999-04-12,0.50,NOK,NaN
2,NaN,1998-04-16,NaN,1998-04-16,0.50,NOK,NaN
0,2020-08-10,2020-08-10,1.0,2020-08-10,25.00,TOPS,25.0


In [37]:
spl_col = list(splits_df.columns)
divd_cols = list(dividend_df.columns)

for i,s in enumerate(spl_col):
    spl_col[i] = "split_"+s
    
for i,s in enumerate(divd_cols):
    divd_cols[i] = "dividend_"+s

In [38]:
spl_df_copy = splits_df.copy()
divd_df_copy = dividend_df.copy()

In [39]:
spl_df_copy.columns = spl_col
divd_df_copy.columns = divd_cols

In [40]:
spl_div = pd.concat([spl_df_copy, divd_df_copy])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [42]:
spl_div.to_csv('split_and_divided_top_5.csv')

In [33]:
ouut = update_all_files()

In [13]:
ouut.tail()

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,post_std_price,post_volume,pre_max_price,pre_median_price,pre_min_price,pre_std_price,pre_volume,symbol,trade_date,group
random_id,,,,,,,,,,,,,,,,,,,,,
-2208491376241434500,91,155.58,200519,16:03:25.863488,157.035,155.76,154.25,0.514818,427176,156.20,...,0.323604,952,156.38,155.34,153.00,0.791475,782,ZTS,2021-02-01,5
-344334069481381418,92,182.91,106492,16:00:02.148791,183.030,181.30,178.51,0.966177,353152,178.51,...,0.152694,720,183.80,176.52,176.52,1.431403,2544,GPN,2021-02-01,5
1296573678968209987,93,148.04,185532,16:00:01.871703,148.380,147.78,146.18,0.356090,407376,147.01,...,0.165200,768,146.03,145.67,137.23,2.565795,176,CB,2021-02-01,5
6641619533634314221,94,321.84,127514,16:00:02.764272,322.890,321.77,317.18,1.110102,545456,319.75,...,0.330388,912,320.50,317.00,317.00,0.824380,1488,SPGI,2021-02-01,5
-7224125209045809356,95,206.68,315043,16:03:54.904029,209.130,206.84,203.26,0.863712,522304,204.22,...,0.151002,576,204.22,203.43,203.10,0.575529,48,AON,2021-02-01,5


In [34]:
ouut.to_csv('data_only_group_5.csv')

In [16]:
all_etf_files = os.listdir('pv_adv_major_etfs')

In [19]:
all_etf_files_df = update_all_etf_files()

In [20]:
all_etf_files_df.head()

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,post_min_price,post_std_price,post_volume,pre_max_price,pre_median_price,pre_min_price,pre_std_price,pre_volume,symbol,trade_date
random_id,,,,,,,,,,,,,,,,,,,,,
-516833153853693080,0,312.96,644326,16:00:00.516757,315.64,312.670,306.9056,1.602895,13032676,315.56,...,306.9056,0.389321,230061,316.9700,315.15,300.74,1.864611,484534,SPY,2020-06-16
-586223318467201895,1,242.85,258565,16:00:00.484560,245.69,242.620,238.5969,1.023006,5575524,243.69,...,238.5969,0.328995,59432,244.3000,243.62,238.75,0.913363,95149,QQQ,2020-06-16
-3486786544043964434,2,55.44,304807,16:00:00.247182,56.11,55.400,54.6500,0.256568,341472,56.09,...,54.9200,0.173766,320,56.1900,56.08,52.94,0.683092,336,XLC,2020-06-16
-7388027181577335645,3,129.19,44716,16:00:00.499798,130.54,128.960,126.5300,0.580210,914784,130.38,...,127.7800,0.510001,544,130.9299,130.21,126.53,0.900061,1328,XLY,2020-06-16
-8274162095056420467,4,59.71,191308,16:00:00.040780,60.02,59.665,58.7900,0.213217,864032,59.95,...,59.5200,0.065733,224,60.0000,59.44,58.79,0.364350,448,XLP,2020-06-16


In [24]:
etf_cols = list(all_etf_files_df.columns)

for i,s in enumerate(etf_cols):
    etf_cols[i] = "etf_"+s

In [25]:
etf_df_copy = all_etf_files_df.copy()
etf_df_copy.columns = etf_cols

In [26]:
etf_df_copy.head()

,etf_Unnamed: 0,etf_close_price,etf_close_size,etf_close_time,etf_intraday_max_price,etf_intraday_median_price,etf_intraday_min_price,etf_intraday_std_price,etf_intraday_volume,etf_open_price,...,etf_post_min_price,etf_post_std_price,etf_post_volume,etf_pre_max_price,etf_pre_median_price,etf_pre_min_price,etf_pre_std_price,etf_pre_volume,etf_symbol,etf_trade_date
random_id,,,,,,,,,,,,,,,,,,,,,
-516833153853693080,0,312.96,644326,16:00:00.516757,315.64,312.670,306.9056,1.602895,13032676,315.56,...,306.9056,0.389321,230061,316.9700,315.15,300.74,1.864611,484534,SPY,2020-06-16
-586223318467201895,1,242.85,258565,16:00:00.484560,245.69,242.620,238.5969,1.023006,5575524,243.69,...,238.5969,0.328995,59432,244.3000,243.62,238.75,0.913363,95149,QQQ,2020-06-16
-3486786544043964434,2,55.44,304807,16:00:00.247182,56.11,55.400,54.6500,0.256568,341472,56.09,...,54.9200,0.173766,320,56.1900,56.08,52.94,0.683092,336,XLC,2020-06-16
-7388027181577335645,3,129.19,44716,16:00:00.499798,130.54,128.960,126.5300,0.580210,914784,130.38,...,127.7800,0.510001,544,130.9299,130.21,126.53,0.900061,1328,XLY,2020-06-16
-8274162095056420467,4,59.71,191308,16:00:00.040780,60.02,59.665,58.7900,0.213217,864032,59.95,...,59.5200,0.065733,224,60.0000,59.44,58.79,0.364350,448,XLP,2020-06-16


In [27]:
etf_df_copy.to_csv('etf_data_5_groups.csv')

In [ ]:
for index, row in tqdm(df.iterrows(), desc='Symbols', total=len(df)):
    
    for item in list(row['splits']):
        ex_date = item['exDate']

        payment_date = item['paymentDate']
        ratio = item['ratio']
        symbol = item['ticker']

        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'split_ex_date'] = ex_date
        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'split_ratio'] = ratio

In [ ]:
for index, row in tqdm(all_etf_files_df.iterrows(), desc='Symbols', total=len(all_etf_files_df)):

    for colname in all_etf_files_df.columns:
        
        new_new_file.loc[(new_new_file['trade_date'] == row['trade_date']), row['symbol']+'_'+colname] = row[colname]



In [ ]:
for index, row in tqdm(symbols_with_dividends.iterrows(), desc='Symbols', total=len(symbols_with_dividends)):
    
    for item in list(row['dividends']):
        ex_date = item['exDate']

        payment_date = item['paymentDate']
        ratio = item['amount']
        symbol = item['ticker']

        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'div_ex_date'] = ex_date
        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'dividend_ratio'] = ratio

In [ ]:
res = str(splits_data['splits'][1]).strip('][').split(', ')